In [2]:
import numpy as np
from data import dataset
import pandas as pd
from abc import ABCMeta, abstractmethod
import pickle
#https://github.com/toxtli/lenet-5-mnist-from-scratch-numpy

In [50]:
class FC():
    """
    Fully connected layer
    """
    def __init__(self, D_in, D_out):
        #print("Build FC")
        self.cache = None
        #self.W = {'val': np.random.randn(D_in, D_out), 'grad': 0}
        self.W = {'val': np.random.normal(0.0, np.sqrt(2/D_in), (D_in,D_out)), 'grad': 0}
        self.b = {'val': np.random.randn(D_out), 'grad': 0}

    def _forward(self, X):
        #print("FC: _forward")
        out = np.dot(X, self.W['val']) + self.b['val']
        self.cache = X
        return out

    def _backward(self, dout):
        #print("FC: _backward")
        X = self.cache
        dX = np.dot(dout, self.W['val'].T).reshape(X.shape)
        self.W['grad'] = np.dot(X.reshape(X.shape[0], np.prod(X.shape[1:])).T, dout)
        self.b['grad'] = np.sum(dout, axis=0)
        #self._update_params()
        return dX

    def _update_params(self, lr=0.001):
        # Update the parameters
        self.W['val'] -= lr*self.W['grad']
        self.b['val'] -= lr*self.b['grad']
class ReLU():
    """
    ReLU activation layer
    """
    def __init__(self):
        #print("Build ReLU")
        self.cache = None

    def _forward(self, X):
        #print("ReLU: _forward")
        out = np.maximum(0, X)
        self.cache = X
        return out

    def _backward(self, dout):
        #print("ReLU: _backward")
        X = self.cache
        dX = np.array(dout, copy=True)
        dX[X <= 0] = 0
        return dX
class Softmax():
    """
    Softmax activation layer
    """
    def __init__(self):
        #print("Build Softmax")
        self.cache = None

    def _forward(self, X):
        #print("Softmax: _forward")
        maxes = np.amax(X, axis=1)
        maxes = maxes.reshape(maxes.shape[0], 1)
        Y = np.exp(X - maxes)
        Z = Y / np.sum(Y, axis=1).reshape(Y.shape[0], 1)
        self.cache = (X, Y, Z)
        return Z # distribution

    def _backward(self, dout):
        X, Y, Z = self.cache
        dZ = np.zeros(X.shape)
        dY = np.zeros(X.shape)
        dX = np.zeros(X.shape)
        N = X.shape[0]
        for n in range(N):
            i = np.argmax(Z[n])
            dZ[n,:] = np.diag(Z[n]) - np.outer(Z[n],Z[n])
            M = np.zeros((N,N))
            M[:,i] = 1
            dY[n,:] = np.eye(N) - M
        dX = np.dot(dout,dZ)
        dX = np.dot(dX,dY)
        return dX
class SGD():
    def __init__(self, params, lr=0.001, reg=0):
        self.parameters = params
        self.lr = lr
        self.reg = reg

    def step(self):
        for param in self.parameters:
            param['val'] -= (self.lr*param['grad'] + self.reg*param['val'])
class Net(metaclass=ABCMeta):
    # Neural network super class

    @abstractmethod
    def __init__(self):
        pass

    @abstractmethod
    def forward(self, X):
        pass

    @abstractmethod
    def backward(self, dout):
        pass

    @abstractmethod
    def get_params(self):
        pass

    @abstractmethod
    def set_params(self, params):
        pass
class NN(Net):

    def __init__(self,in_,mid_,weights=''):

        self.FC1 = FC(in_, mid_)
        self.FC2 = FC(mid_, mid_)
        self.FC3=FC(mid_, 2)
        self.relu=ReLU()
        self.Softmax = Softmax()
        if weights == '':
            pass
        else:
            with open(weights,'rb') as f:
                params = pickle.load(f)
                self.set_params(params)

    def forward(self, x):

        x=self.FC1._forward(x)
        x=self.relu._forward(x)
        x= self.FC2._forward(x)
        x=self.relu._forward(x)
        x= self.FC3._forward(x)
        x= self.Softmax._forward(x)
        return x

    def backward(self, dout):
        dout = self.FC3._backward(dout)
        dout = self.relu._backward(dout)
        dout = self.FC2._backward(dout)
        dout = self.relu._backward(dout)
        dout = self.FC2._backward(dout)
        
    def get_params(self):
        return [self.FC1.W, self.FC1.b, self.FC2.W, self.FC2.b,self.FC3.W, self.FC3.b]

    def set_params(self, params):
        [self.FC1.W, self.FC1.b, self.FC2.W, self.FC2.b,self.FC3.W, self.FC3.b] = params


In [37]:
def MakeOneHot(Y, D_out=2):
    N = Y.shape[0]
    Z = np.zeros((N, D_out))
    Z[np.arange(N), Y] = 1
    return Z

In [67]:
##這裡要改路徑
data=dataset('C:\\Users\\Robert\\Desktop\\hw2\\train.csv')
data.train_data[0].shape

(42,)

In [117]:
def model_train(model,select_feature,epoch=20,data=data.train_data,label=data.train_label):
    optim = SGD(model.get_params())
    for i in range(epoch):
        for index, (row,labels) in enumerate(zip(data,label)):
            row=np.expand_dims(row,axis=0)
            labels=np.expand_dims(labels,axis=0)
            labels=MakeOneHot(labels,2)
            Y_pred = model.forward(row[:,select_feature])
            #先算到softmax的back
            dout=Y_pred-labels
            model.backward(dout)
            optim.step()
    acc=0
    for index, (row,labels) in enumerate(zip(data,label)):
        row=np.expand_dims(row,axis=0)
        Y_pred = model.forward(row[:,select_feature])
        Y_pred=np.argmax(Y_pred,1)
        if Y_pred[0]==labels:
            acc+=1
    print(f"Accuracy: {acc/(len(label))}")

In [92]:
def model_test(model,select_feature,data=data,label=data):
    acc=0
    for index, (row,labels) in enumerate(zip(data,label)):
        row=np.expand_dims(row,axis=0)
        Y_pred = model.forward(row[:,select_feature])
        Y_pred=np.argmax(Y_pred,1)
        if Y_pred[0]==labels:
            acc+=1
    print(f"Accuracy: {acc/(len(label))}")

In [69]:
import random
numbers = list(range(1, 42))

In [118]:
nn1=NN(10,20)
selected_1 = random.sample(numbers, 10)
model_train(nn1,selected_1)


nn2=NN(10,20)
selected_2 = random.sample(numbers, 10)
model_train(nn2,selected_2)

nn3=NN(10,20)
selected_3 = random.sample(numbers, 10)
model_train(nn1,selected_3)

nn4=NN(10,20)
selected_4 = random.sample(numbers, 10)
model_train(nn1,selected_4)

nn5=NN(10,20)
selected_5 = random.sample(numbers, 10)
model_train(nn1,selected_5)

Accuracy: 0.948
Accuracy: 0.948
Accuracy: 0.948
Accuracy: 0.948
Accuracy: 0.948


In [119]:
model_test(nn1,selected_1,data.val_data,data.val_label)
model_test(nn2,selected_2,data.val_data,data.val_label)
model_test(nn3,selected_3,data.val_data,data.val_label)
model_test(nn4,selected_4,data.val_data,data.val_label)
model_test(nn5,selected_5,data.val_data,data.val_label)

Accuracy: 0.9380440348182284
Accuracy: 0.9380440348182284
Accuracy: 0.061955965181771634
Accuracy: 0.6033452807646356
Accuracy: 0.844342037890425


In [120]:
model_test(nn1,selected_1,data.test_data,data.test_label)
model_test(nn2,selected_2,data.test_data,data.test_label)
model_test(nn3,selected_3,data.test_data,data.test_label)
model_test(nn4,selected_4,data.test_data,data.test_label)
model_test(nn5,selected_5,data.test_data,data.test_label)

Accuracy: 0.9368600682593856
Accuracy: 0.9368600682593856
Accuracy: 0.06313993174061433
Accuracy: 0.606655290102389
Accuracy: 0.8435153583617747


In [121]:
def combine_test(model1,model2,model3,model4,model5,selected_1,selected_2,selected_3,selected_4,selected_5,data,label):
    acc=0
    for index, (row,labels) in enumerate(zip(data,label)):
        Y_pred=[]
        for i in range(1,6):
            row_i=np.expand_dims(row,axis=0)
            model='model'+str(i)
            select='selected_'+str(i)
            pred = eval(model).forward(row_i[:,eval(select)])
            Y_pred.append(np.argmax(pred,1)[0])
        Y_pred=int(sum(Y_pred)/len(Y_pred))
        if Y_pred==labels:
            acc+=1
    print(f"Accuracy: {acc/(len(label))}")

In [122]:
combine_test(nn1,nn2,nn3,nn4,nn5,selected_1,selected_2,selected_3,selected_4,selected_5,data.train_data,data.train_label)
combine_test(nn1,nn2,nn3,nn4,nn5,selected_1,selected_2,selected_3,selected_4,selected_5,data.val_data,data.val_label)
combine_test(nn1,nn2,nn3,nn4,nn5,selected_1,selected_2,selected_3,selected_4,selected_5,data.test_data,data.test_label)

Accuracy: 0.948
Accuracy: 0.9380440348182284
Accuracy: 0.9368600682593856
